# 1.Getting the data ready to be used with ML model

Three main things to do:

1. Split the data into features and labels (usually `X` & `y`)
2. Filling (Imputing) or disregarding missing values
3. Converting non-numerical values into numerical values (feature encoding)

In [2]:
# Importing Essentials
import numpy as np
import pandas as pd

In [61]:
# Loading the data
car_sales_missing = pd.read_csv('data/car-sales-extended-missing-data.csv')

In [62]:
#getting familiar with the data
car_sales_missing

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


In [63]:
# Function to display all the rows and columns as sometimes they are omitted in the notebook: 

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

## 1.1 Dealing with missing values

In [64]:
# Checking the missing values in data
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [65]:
# Percantage of missing data in each column
car_sales_missing.isna().sum()/car_sales_missing.shape[0]

Make             0.049
Colour           0.050
Odometer (KM)    0.050
Doors            0.050
Price            0.050
dtype: float64

In [66]:
# Drop the rows with no labels, i.e drop the rows where the value in target variable is nan
car_sales_missing.dropna(subset=['Price'], inplace = True)

## There are 2 ways to deal with missinge values

### Option 1: Fill missing values (Imputation)

- Numerical data can be replaced with mean, mode or median depending on the requirement and type of data
- Categorical data can be replaced with `missing` which will act as another category
- Generally the missing values are replaced in feature variables, and the rows where there exists a nan value in target variable is dropped
- While replacing the missing values it should be noted that the missing values in training and test sets should be replaced seperately (Data from the future (Test data) should not be used to replace values in the past (Training data) ) 

### Option 2: Remove the rows and columns with null values

- Remove all the rows with even a single null value, since it is not possible for a ML model to process null value
- Remove the columns which has more null values than a given threshold (The threshold may vary for each dataset)

### Filling missing values

#### Dealing with missing values in categorical features
Replacing nan values in categorical features with `missing`, making it another category.

Even if we don't replace the missing values in categorical features, the code (`ML.txt` line 39) will replace the nan values with -1.

In [67]:
car_sales_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Make           903 non-null    object 
 1   Colour         904 non-null    object 
 2   Odometer (KM)  902 non-null    float64
 3   Doors          903 non-null    float64
 4   Price          950 non-null    float64
dtypes: float64(3), object(2)
memory usage: 44.5+ KB


In [68]:
car_sales_missing['Make'].fillna('missing', inplace = True)
car_sales_missing['Colour'].fillna('missing', inplace = True)

In [69]:
# Converting features with object dtype to categorical features
categorical = []
for col in car_sales_missing.columns[:]:
    if car_sales_missing[col].dtype == 'object' : categorical.append(col)  # pandas treat "str" as "object"
categorical # list of all the variables which are strings

['Make', 'Colour']

In [72]:
for col in categorical: car_sales_missing[col] = car_sales_missing[col].astype("category").cat.as_ordered()
for col in categorical : car_sales_missing[col] = car_sales_missing[col].cat.codes

In [73]:
# Replacing missing values in `Doors` column with the mode
car_sales_missing['Doors'].mode()

0    4.0
dtype: float64

In [74]:
car_sales_missing['Doors'].fillna(4, inplace = True)

#### Dealing with missing values in numerical features

To fill the missing numerical data the dataset is first split into train and test and then the missing values are filled individually

In [75]:
from sklearn.model_selection import train_test_split

# Split into feature and target variables
X = car_sales_missing.drop('Price', axis=1)
y = car_sales_missing['Price']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2)

In [76]:
# Checking the shape of train and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((760, 4), (190, 4), (760,), (190,))

In [77]:
# Replacing nan values in 'Odometer (KM)' column with the average of other values in the column
X_train['Odometer (KM)'].fillna(X_train['Odometer (KM)'].mean(), inplace = True)
X_test['Odometer (KM)'].fillna(X_test['Odometer (KM)'].mean(), inplace = True)

C:\Users\KIIT\Desktop\sample_project_1\env\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [78]:
# Checking the count of nan values in train and test data
X_train.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64

In [79]:
X_test.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64

In [110]:
# Checking the dtype of columns
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 347 to 715
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Make           760 non-null    int8   
 1   Colour         760 non-null    int8   
 2   Odometer (KM)  760 non-null    float64
 3   Doors          760 non-null    float64
dtypes: float64(2), int8(2)
memory usage: 19.3 KB


## Working with missing values using Scikit-Learn

In [3]:
car_sales_missing = pd.read_csv('data/car-sales-extended-missing-data.csv')

In [4]:
car_sales_missing.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [5]:
# Check missing values
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [6]:
# Drop the rows with no labels
car_sales_missing.dropna(subset=["Price"], inplace=True)
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [7]:
# Dividing the data into training and test sets
from sklearn.model_selection import train_test_split

# Split into X & y
X = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

# Split data into train and test
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2)

Filling the training and test values separately to ensure training data stays with the training data and test data stays with the test data.

Note: We use fit_transform() on the training data and transform() on the testing data. In essence, we learn the patterns in the training set and transform it via imputation (fit, then transform). Then we take those same patterns and fill the test set (transform only).

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Filling categorical values with 'missing' and numerical values with mean
cat_imputer = SimpleImputer(strategy = 'constant', fill_value = 'missing')
door_imputer = SimpleImputer(strategy = 'constant',  fill_value = 4)
num_imputer = SimpleImputer(strategy = 'mean')

# Define columns
cat_columns = ["Make", "Colour"]
door_columns = ["Doors"]
num_columns = ["Odometer (KM)"]

# Create an imputer: Something that fills the missing values
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_columns),
    ("door_imputer", door_imputer, door_columns),
    ("num_imputer", num_imputer, num_columns)
])

# Filling the train and test sets seperately
filled_X_train = imputer.fit_transform(X_train) # fit_transform imputes the missing values from the training set and fills them simultaneously
filled_X_test = imputer.transform(X_test) # tranform takes the imputing missing values from the training set and fills the test set with them

# Check filled X_train
filled_X_train

array([['Honda', 'White', 4.0, 71934.0],
       ['Toyota', 'Red', 4.0, 162665.0],
       ['Honda', 'White', 4.0, 42844.0],
       ...,
       ['Toyota', 'White', 4.0, 196225.0],
       ['Honda', 'Blue', 4.0, 133117.0],
       ['Honda', 'missing', 4.0, 150582.0]], dtype=object)

In [11]:
# Checking the missing values in new datasets

# Get our transformed data array's back into DataFrame's
car_sales_filled_train = pd.DataFrame(filled_X_train, 
                                      columns=["Make", "Colour", "Doors", "Odometer (KM)"])

car_sales_filled_test = pd.DataFrame(filled_X_test, 
                                     columns=["Make", "Colour", "Doors", "Odometer (KM)"])

# Check missing data in training set
car_sales_filled_train.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [12]:
# Check missing data in test set
car_sales_filled_test.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

So the data has no missing values, but still we need to convert some of the data into numbers, so using one hot encoding.
Again keeping the training and testing datasets seperate

In [13]:
# Import OneHotEncoder class from sklearn
from sklearn.preprocessing import OneHotEncoder

# one hot encode the features
categorical_features = ["Make", "Colour", "Doors"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", 
                                 one_hot, 
                                 categorical_features)],
                                 remainder="passthrough")

# Fill train and test values separately
transformed_X_train = transformer.fit_transform(car_sales_filled_train) # fit and transform the training data
transformed_X_test = transformer.transform(car_sales_filled_test) # transform the test data

# Check transformed and filled X_train
transformed_X_train.toarray()

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 7.19340e+04],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.62665e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 4.28440e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.96225e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.33117e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.50582e+05]])

## Fit a model

The data is finally ready and can be fitted in a model

In [14]:
# Now we've transformed X, let's see if we can fit a model
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor

# Setup model
model = RandomForestRegressor()

# Make sure to use transformed (filled and one-hot encoded X data)
model.fit(transformed_X_train, y_train)

# Evaluating on the test data
model.score(transformed_X_test, y_test)

0.21229043336119102